In [1]:
!pip install polaris-lib

In [2]:
!polaris login

2024-06-20 15:27:59.996 | INFO     | polaris.hub.client:login:285 - You are already logged in to the Polaris Hub as  (andrea.roncoli.cs@gmail.com). Set `overwrite=True` to force re-authentication.


In [2]:
import polaris as po
import datamol as dm
import numpy as np

benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
train[0]

2024-06-20 19:24:56.839 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-20 19:24:56.843 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


(array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 {'CLASS_KIT_(T6701_mutant)': 0.0,
  'CLASS_KIT_(V560G_mutant)': 0.0,
  'CLASS_KIT': 0.0})

In [4]:

from rdkit import Chem

def canonicalize_smiles(smiles: str) -> str:
    try:
        # Convert SMILES string to a molecule
        molecule = Chem.MolFromSmiles(smiles)
        if molecule:
            # Generate canonical SMILES
            canonical_smiles = Chem.MolToSmiles(molecule, canonical=True)
            return canonical_smiles
        else:
            return "Invalid SMILES string."
    except Exception as e:
        return f"Error: {e}"

# Example usage
smiles = "COc1cccc(COc2cc(sc2C(N)=O)-n2cnc3ccccc23)c1"
canonical_smiles = canonicalize_smiles(smiles)
print(f"Original SMILES: {smiles}")
print(f"Canonical SMILES: {canonical_smiles}")

Original SMILES: COc1cccc(COc2cc(sc2C(N)=O)-n2cnc3ccccc23)c1
Canonical SMILES: COc1cccc(COc2cc(-n3cnc4ccccc43)sc2C(N)=O)c1


In [8]:
from rdkit import Chem
import pandas as pd
file_path = 'FULL CSV DATA.csv'
df = pd.read_csv(file_path)
# Function to canonicalize SMILES
def canonicalize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True)
        else:
            return None
    except:
        return None


# Apply the canonicalization
df['Canonical_SMILES'] = df['Compound ID'].apply(canonicalize_smiles)

# Save the updated dataframe
df.to_csv('canonicalized_smiles_data.csv', index=False)

In [9]:
# Compare original and canonicalized SMILES
df['Changed'] = df['Compound ID'] != df['Canonical_SMILES']

# Count the number of changed SMILES
num_changed = df['Changed'].sum()

# Save the updated dataframe with comparisons
df.to_csv('canonicalized_smiles_comparison.csv', index=False)

print(f"Number of SMILES canonicalized: {num_changed}")

Number of SMILES canonicalized: 355


In [8]:
ys = train.y
ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
ys.shape
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()

276

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=5)
model.fit(train.X[mask], ys[mask])
y_pred = model.predict(test.X)
y_pred.shape

(87, 3)

In [15]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 10.8 MB/s eta 0:00:0000:0100:01


In [22]:
from xgboost import XGBClassifier

# Initialize the XGBoost model
xgb_model = XGBClassifier(max_depth=5)

# Fit the model on the training data
xgb_model.fit(train.X[mask], ys[mask])

# Predict on the test data
xgb_y_pred = xgb_model.predict(test.X)

# Predict probabilities on the test data
xgb_y_prob = xgb_model.predict_proba(test.X)

# Assuming benchmark.target_cols is a list of target column names
# Ensure y_pred and y_prob have the expected structure
xgb_y_pred = {k: (xgb_y_pred == idx).astype(int) for idx, k in enumerate(benchmark.target_cols)}

# Stack probabilities and extract the required columns
# We assume a binary classification for each target column (hence using index 1 for probabilities)
xgb_y_prob = np.stack(xgb_y_prob, axis=1)
xgb_y_prob = {k: xgb_y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}

print(xgb_y_pred)
print(xgb_y_prob)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [24]:
from xgboost import XGBClassifier
import numpy as np

# Create a dictionary of XGBoost models, one for each target column
models = {target: XGBClassifier(max_depth=5, use_label_encoder=False, eval_metric='logloss') for target in benchmark.target_cols}
X = train.X

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(test.X)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(test.X) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:50:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Shapes of y_pred: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}
Shapes of y_prob: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}


In [21]:
from xgboost import XGBClassifier
import numpy as np

# Create a dictionary of XGBoost models, one for each target column
models = {target: XGBClassifier(max_depth=5, use_label_encoder=False, eval_metric='logloss') for target in benchmark.target_cols}
X = train.X

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(test.X)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(test.X) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})


results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)
print(results)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:59:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Shapes of y_pred: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}
Shapes of y_prob: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}
name=None description='' tags=[] user_attributes={} owner=None polaris_version='dev' results=   Test set              Target label              Metric     Score
0      test  CLASS_KIT_(T6701_mutant)     Metric.accuracy  0.908046
1      test  CLASS_KIT_(V560G_mutant)     Metric.accuracy  0.862069
2      test                 CLASS_KIT     Metric.accuracy  0.678161
3      test  CLASS_KIT_(T6701_mutant)           Metric.f1  0.600000
4      test  CLASS_KIT_(V560G_mutant)           Metric.f1  0.142857
5      test                 CLASS_KIT           Metric.f1  0.416667
6      test  CLASS_KIT_(T6701_mutant)      Metric.roc_auc  0.724560
7      test  CLASS_KIT_(V560G_mutant)      Metric.roc_auc  0.771111
8      test                 CLASS_KIT      Metric.roc_auc  0.753648
9      te

In [24]:
from xgboost import XGBClassifier
import numpy as np

# Create a dictionary of XGBoost models, one for each target column
models = {target: XGBClassifier(max_depth=5, use_label_encoder=False, eval_metric='auc', objective='binary:logistic') for target in benchmark.target_cols}
X = train.X

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(test.X)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(test.X) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})


results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)
results.name = "xgb-auc-binarylog-for-loop"
results.description = "xgb For Loop Based on Notebook"
results.upload_to_hub(owner="team13")

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:04:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Shapes of y_pred: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}
Shapes of y_prob: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}


2024-06-20 21:04:24.339 | SUCCESS  | polaris.hub.client:upload_results:492 - Your result has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/polaris/pkis1-kit-wt-mut-c-1/0t1MtZlh5xFmdDx5wnfl6


{'id': '0t1MtZlh5xFmdDx5wnfl6',
 'createdAt': '2024-06-20T21:04:24.314Z',
 'deletedAt': None,
 'name': 'xgb-mae-binarylog-for-loop',
 'slug': 'xgb-mae-binarylog-for-loop',
 'description': 'xgb For Loop Based on Notebook',
 'tags': [],
 'userAttributes': {},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'ownerId': 'n7RilKwWATJHjBVq7pcA9',
 'creatorId': 'pXNk8yXN524Fm9fPLMrbe',
 'benchmarkId': 'DZzlykxvBwlSA9uERL17A',
 'results': [{'scores': {'f1': 0.4166666666666667,
    'mcc': 0.2703121333894177,
    'pr_auc': 0.6477497857237808,
    'roc_auc': 0.7536475869809203,
    'accuracy': 0.6781609195402298,
    'cohen_kappa': 0.2354048964218456},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT'},
  {'scores': {'f1': 0.6,
    'mcc': 0.6214848238238696,
    'pr_auc': 0.6636492977388023,
    'roc_auc': 0.7245596868884541,
    'accuracy': 0.9080459770114943,
    'cohen_kappa': 0.5572519083969466},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT_(T6701_mutant)'},
  {

In [17]:
import os
import tqdm
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
import os
import tqdm.auto as tqdm
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer, log_loss, accuracy_score
from xgboost import XGBClassifier

def mkdirs(path : str):
    """Create a directory if it does not exist."""

    if not os.path.exists(path) : os.makedirs(path)

def compute_fps(data):
    """Compute Morgan Fingerprints from SMILES."""

    fps = pd.DataFrame(np.array([AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 3, nBits=2048) for smiles in tqdm.tqdm(data.SMILES, desc='Computing Morgan Fingerprints from SMILES')]), index=data.index)

    return fps

def XGB_hyperparams():
    """Returns a dictionary of hyperparameters for XGBoost"""
    return {
        "max_depth": [4, 5, 6, 7, 8],
        "learning_rate": [0.01, 0.05, 0.1, 0.2],
        "n_estimators": [100, 200, 300, 400],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "subsample": [0.6, 0.8, 1.0],
        "gamma": [0, 0.1, 0.2, 0.5],
        "reg_alpha": [0, 0.01, 0.1, 1],
        "reg_lambda": [1, 1.5, 2],
    }

def train_optim_XGB_STs(data_path, val_path, model_path, seed=2022, n_jobs=16, n_iter=50):
    """Trains XGBoost models for classification tasks using Randomized Search CV"""
    mkdirs(model_path)

    data = pd.read_csv(data_path)
    data['split_index'] = -1

    val = pd.read_csv(val_path)
    val['split_index'] = 0

    combined_data = pd.concat([data, val], axis=0, ignore_index=True)
    targets = [col for col in combined_data.columns if col not in ['Canonical_SMILES', 'Compound ID' , 'Comp Conc. (µM)' , 'Chemotype','Pubmed ID', 'Split', 'Subset', 'set_index']]

    fps = compute_fps(combined_data, radius=3, nbits=2048)

    for target in tqdm.tqdm(targets, desc='Training XGB models'):
        if os.path.exists(f'{model_path}/{target}.joblib'):
            continue
        
        y = combined_data[target].dropna()
        X = fps.iloc[y.index]
        splits = PredefinedSplit(test_fold=combined_data.iloc[y.index].split_index)

        model = XGBClassifier(random_state=seed, use_label_encoder=False, tree_method='gpu_hist')
        rdnSearch = RandomizedSearchCV(
            model, XGB_hyperparams(), n_iter=n_iter, cv=splits, n_jobs=1,
            scoring=make_scorer(log_loss, needs_proba=True), random_state=seed
        )
        rdnSearch.fit(X, y)
        joblib.dump(rdnSearch.best_estimator_, f'{model_path}/{target}.joblib')
        print(f'Best parameters for {target}: {rdnSearch.best_params_}')
        print(f'Best score for {target}: {rdnSearch.best_score_}')
        print(f'Best model saved to {model_path}/{target}.joblib')

def predict_XGB_STs(data_path, model_path, preds_path=None):
    """Predicts the targets for a dataset using trained XGBoost models"""
    data = pd.read_csv(data_path)
    fps = compute_fps(data, radius=3, nbits=2048)
    targets = [col for col in data.columns if col not in ['Canonical_SMILES', 'Compound ID' , 'Comp Conc. (µM)' , 'Chemotype','Pubmed ID', 'Split', 'Subset', 'set_index']]

    preds = data.copy()

    for target in tqdm.tqdm(targets, desc='Predicting with XGBoosts'):
        model = joblib.load(f'{model_path}/{target}.joblib')
        preds[target] = model.predict(fps)

    if preds_path:
        mkdirs(os.path.dirname(preds_path))
        preds.to_csv(preds_path, index=False)

    return preds

def train_XGB_STs(data_path, model_path, seed=2022, gpu_id=0):
    """Trains XGBoost models for each target without hyperparameter tuning"""
    mkdirs(model_path)

    data = pd.read_csv(data_path)
    targets = [col for col in data.columns if col != 'Canonical_SMILES']

    fps = compute_fps(data, radius=3, nbits=2048)

    for target in tqdm.tqdm(targets, desc='Training XGBoosts'):
        y_train = data[target].dropna()
        X_train = fps.iloc[y_train.index]
        
        model = XGBClassifier(tree_method='gpu_hist', random_state=seed, gpu_id=gpu_id)
        model.fit(X_train, y_train)
        joblib.dump(model, f'{model_path}/{target}.joblib')

# Assuming benchmark is a provided module for evaluating the model
def evaluate_models(models, test_X, test_y):
    """Evaluates models using benchmark"""
    y_prob = {target: model.predict_proba(test_X)[:, 1] for target, model in models.items()}
    y_pred = {target: model.predict(test_X) for target, model in models.items()}

    results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)
    print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
    print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})

    return results

In [25]:
benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.908045977
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6781609195
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.6
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.1428571429
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.4166666667
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7245596869
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7711111111
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.753647587
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6636492977
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4420557438
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6477497857
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.6214848238
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.1610626477
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.2703121334
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.5572519084
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.1076923077
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.2354048964
    }
  ]
}

ValueError: y should be a 1d array, got an array of shape (276, 3) instead.

In [20]:
y_prob = model.predict_proba(test.X)
y_prob = np.stack(y_prob, axis=1)
y_prob.shape

(87, 3, 2)

In [21]:
y_pred = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob = {k: y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}

TypeError: unhashable type: 'slice'

In [14]:
y_pred = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob = {k: y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

In [18]:
benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [33]:
results.name = "my-first-boost"
results.description = "Awal test ma3 el boost el 2adim"
results.upload_to_hub(owner="team13")

2024-06-20 16:00:28.763 | SUCCESS  | polaris.hub.client:upload_results:492 - Your result has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/polaris/pkis1-kit-wt-mut-c-1/SEKGzrlAcYw0k9mTdUPXL


{'id': 'SEKGzrlAcYw0k9mTdUPXL',
 'createdAt': '2024-06-20T16:00:28.737Z',
 'deletedAt': None,
 'name': 'my-first-boost',
 'slug': 'my-first-boost',
 'description': 'Awal test ma3 el boost el 2adim',
 'tags': [],
 'userAttributes': {},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'ownerId': 'n7RilKwWATJHjBVq7pcA9',
 'creatorId': 'pXNk8yXN524Fm9fPLMrbe',
 'benchmarkId': 'DZzlykxvBwlSA9uERL17A',
 'results': [{'scores': {'f1': 0.4166666666666667,
    'mcc': 0.2703121333894177,
    'pr_auc': 0.6477497857237808,
    'roc_auc': 0.7536475869809203,
    'accuracy': 0.6781609195402298,
    'cohen_kappa': 0.2354048964218456},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT'},
  {'scores': {'f1': 0.6,
    'mcc': 0.6214848238238696,
    'pr_auc': 0.6636492977388023,
    'roc_auc': 0.7245596868884541,
    'accuracy': 0.9080459770114943,
    'cohen_kappa': 0.5572519083969466},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT_(T6701_mutant)'},
  {'scores': {'f1': 0.1428

In [35]:
from sklearn.ensemble import RandomForestClassifier

models = {target: RandomForestClassifier(max_depth=5) for target in benchmark.target_cols}
X = train.X

for target, model in models.items():
  y = train.y[target]
  mask = ~np.isnan(y)
  model.fit(X[mask], y[mask])

y_prob = {target: model.predict_proba(test.X)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(test.X) for target, model in models.items()}

results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)
results.name = "my-first-rf-for-loop"
results.description = "RF For Loop Based on Notebook"
results.upload_to_hub(owner="team13")

2024-06-20 16:10:36.097 | SUCCESS  | polaris.hub.client:upload_results:492 - Your result has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/polaris/pkis1-kit-wt-mut-c-1/AE8yDq0bFPm6zbvf5ry10


{'id': 'AE8yDq0bFPm6zbvf5ry10',
 'createdAt': '2024-06-20T16:10:36.070Z',
 'deletedAt': None,
 'name': 'my-first-rf-for-loop',
 'slug': 'my-first-rf-for-loop',
 'description': 'RF For Loop Based on Notebook',
 'tags': [],
 'userAttributes': {},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'ownerId': 'n7RilKwWATJHjBVq7pcA9',
 'creatorId': 'pXNk8yXN524Fm9fPLMrbe',
 'benchmarkId': 'DZzlykxvBwlSA9uERL17A',
 'results': [{'scores': {'f1': 0,
    'mcc': -0.08429675647647886,
    'pr_auc': 0.7035527963473335,
    'roc_auc': 0.82996632996633,
    'accuracy': 0.6091954022988506,
    'cohen_kappa': -0.022821576763485618},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT'},
  {'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.3508688285920429,
    'roc_auc': 0.6908023483365949,
    'accuracy': 0.8390804597701149,
    'cohen_kappa': 0},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT_(T6701_mutant)'},
  {'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.6125134596849

In [36]:
from sklearn.svm import SVC

# Create a dictionary of SVM models, one for each target column
models = {target: SVC(probability=True) for target in benchmark.target_cols}
X = train.X

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(test.X)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(test.X) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})
results.name = "my-first-SVM-for-loop"
results.description = "SVM For Loop Based on Notebook"
results.upload_to_hub(owner="team13")

Shapes of y_pred: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}
Shapes of y_prob: {'CLASS_KIT_(T6701_mutant)': (87,), 'CLASS_KIT_(V560G_mutant)': (87,), 'CLASS_KIT': (87,)}


2024-06-20 16:13:18.731 | SUCCESS  | polaris.hub.client:upload_results:492 - Your result has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/polaris/pkis1-kit-wt-mut-c-1/meBZUFUBPqMQ6EVAi1osd


{'id': 'meBZUFUBPqMQ6EVAi1osd',
 'createdAt': '2024-06-20T16:13:18.704Z',
 'deletedAt': None,
 'name': 'my-first-SVM-for-loop',
 'slug': 'my-first-svm-for-loop',
 'description': 'SVM For Loop Based on Notebook',
 'tags': [],
 'userAttributes': {},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'ownerId': 'n7RilKwWATJHjBVq7pcA9',
 'creatorId': 'pXNk8yXN524Fm9fPLMrbe',
 'benchmarkId': 'DZzlykxvBwlSA9uERL17A',
 'results': [{'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.7060944306375462,
    'roc_auc': 0.781705948372615,
    'accuracy': 0.6206896551724138,
    'cohen_kappa': 0},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT'},
  {'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.6336943478637297,
    'roc_auc': 0.7279843444227005,
    'accuracy': 0.8390804597701149,
    'cohen_kappa': 0},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT_(T6701_mutant)'},
  {'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.40770997612871224,
    'roc_auc': 0.682222222222

In [10]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import pairwise_kernels

def tanimoto_kernel(X, Y=None):
    """
    Compute the Tanimoto kernel between two matrices X and Y.
    """
    if Y is None:
        Y = X
    similarity = np.dot(X, Y.T)
    sum_x = np.sum(X, axis=1)
    sum_y = np.sum(Y, axis=1)
    denominator = sum_x[:, np.newaxis] + sum_y - similarity
    kernel = similarity / denominator
    return kernel

# Create a dictionary of SVM models, one for each target column with custom Tanimoto kernel
models = {target: SVC(kernel='precomputed', probability=True) for target in benchmark.target_cols}
X = train.X

# Compute the kernel matrix for the training data
K_train = tanimoto_kernel(X)

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(K_train[mask][:, mask], y[mask])

# Compute the kernel matrix for the test data
K_test = tanimoto_kernel(test.X, X)

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(K_test)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(K_test) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})
results.name = "my-first-SVM-tanimoto-kernel"
results.description = "SVM Tanimoto Kernel Based on Notebook"
results.upload_to_hub(owner="team13")


ValueError: X has 277 features, but SVC is expecting 276 features as input.

In [11]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import pairwise_kernels

def tanimoto_kernel(X, Y=None):
    """
    Compute the Tanimoto kernel between two matrices X and Y.
    """
    if Y is None:
        Y = X
    similarity = np.dot(X, Y.T)
    sum_x = np.sum(X, axis=1)
    sum_y = np.sum(Y, axis=1)
    denominator = sum_x[:, np.newaxis] + sum_y - similarity
    kernel = similarity / denominator
    return kernel

# Create a dictionary of SVM models, one for each target column with custom Tanimoto kernel
models = {target: SVC(kernel='precomputed', probability=True) for target in benchmark.target_cols}
X = train.X

# Compute the kernel matrix for the training data
K_train = tanimoto_kernel(X)

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(K_train[mask][:, mask], y[mask])

# Compute the kernel matrix for the test data
K_test = tanimoto_kernel(test.X, X)

# Predict probabilities and labels for each target column
y_prob = {target: model.predict_proba(K_test)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(K_test) for target, model in models.items()}

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})
results.name = "my-first-SVM-tanimoto-kernel"
results.description = "SVM Tanimoto Kernel Based on Notebook"
results.upload_to_hub(owner="team13")


ValueError: X has 277 features, but SVC is expecting 276 features as input.

In [12]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import pairwise_kernels

def tanimoto_kernel(X, Y):
    """
    Compute the Tanimoto kernel between two matrices X and Y.
    """
    similarity = np.dot(X, Y.T)
    sum_x = np.sum(X, axis=1)
    sum_y = np.sum(Y, axis=1)
    denominator = sum_x[:, np.newaxis] + sum_y - similarity
    kernel = similarity / denominator
    return kernel

class TanimotoSVC(SVC):
    """
    SVC with a Tanimoto kernel.
    """
    def __init__(self, probability=True, **kwargs):
        super().__init__(kernel='precomputed', probability=probability, **kwargs)
    
    def fit(self, X, y):
        self._X_fit = X  # Save training data for use in kernel computation
        K_train = self._compute_kernel(X)
        super().fit(K_train, y)
        return self
    
    def predict(self, X):
        K_test = self._compute_kernel(X, self._X_fit)
        return super().predict(K_test)
    
    def predict_proba(self, X):
        K_test = self._compute_kernel(X, self._X_fit)
        return super().predict_proba(K_test)
    
    def _compute_kernel(self, X, Y=None):
        if Y is None:
            Y = self._X_fit
        return tanimoto_kernel(X, Y)

# Create a dictionary of SVM models, one for each target column
models = {target: TanimotoSVC() for target in benchmark.target_cols}
X = train.X

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])

# Predict probabilities and labels for each target column
y_prob = {}
y_pred = {}

# Make predictions for each model
for target, model in models.items():
    y_prob[target] = model.predict_proba(test.X)[:, 1]
    y_pred[target] = model.predict(test.X)

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})
results.name = "my-first-SVM-tanimoto-kernel-trick"
results.description = "SVM Tanimoto Kernel with Kernel Trick"
results.upload_to_hub(owner="team13")


ValueError: shapes (87,277) and (2048,277) not aligned: 277 (dim 1) != 2048 (dim 0)

In [13]:
import numpy as np
from sklearn.svm import SVC

def tanimoto_kernel(X, Y=None):
    """
    Compute the Tanimoto kernel between two matrices X and Y.
    If Y is None, compute the kernel between X and itself.
    """
    if Y is None:
        Y = X
    similarity = np.dot(X, Y.T)
    sum_x = np.sum(X, axis=1)
    sum_y = np.sum(Y, axis=1)
    denominator = sum_x[:, np.newaxis] + sum_y - similarity
    kernel = similarity / denominator
    return kernel

class TanimotoSVC(SVC):
    """
    SVC with a Tanimoto kernel.
    """
    def __init__(self, probability=True, **kwargs):
        super().__init__(kernel='precomputed', probability=probability, **kwargs)
    
    def fit(self, X, y):
        self._X_fit = X  # Save training data for use in kernel computation
        K_train = self._compute_kernel(X)
        super().fit(K_train, y)
        return self
    
    def predict(self, X):
        K_test = self._compute_kernel(X, self._X_fit)
        return super().predict(K_test)
    
    def predict_proba(self, X):
        K_test = self._compute_kernel(X, self._X_fit)
        return super().predict_proba(K_test)
    
    def _compute_kernel(self, X, Y=None):
        if Y is None:
            Y = self._X_fit
        return tanimoto_kernel(X, Y)

# Create a dictionary of SVM models, one for each target column
models = {target: TanimotoSVC() for target in benchmark.target_cols}
X = train.X

# Compute the Tanimoto kernel matrix for the training data
K_train = tanimoto_kernel(X)

# Fit each model on the corresponding target column
for target, model in models.items():
    y = train.y[target]
    mask = ~np.isnan(y)  # Filter out rows where the target value is NaN
    model.fit(X[mask], y[mask])  # X[mask] is passed for consistency, the class handles the kernel

# Compute the Tanimoto kernel matrix for the test data relative to the training data
K_test = tanimoto_kernel(test.X, X)

# Predict probabilities and labels for each target column
y_prob = {}
y_pred = {}

# Make predictions for each model
for target, model in models.items():
    y_prob[target] = model.predict_proba(test.X)[:, 1]
    y_pred[target] = model.predict(test.X)

# Evaluate results
results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

# Print shapes of predictions and probabilities for debugging
print("Shapes of y_pred:", {target: pred.shape for target, pred in y_pred.items()})
print("Shapes of y_prob:", {target: prob.shape for target, prob in y_prob.items()})
results.name = "my-first-SVM-tanimoto-kernel-compute"
results.description = "SVM Tanimoto Kernel Computation for Each Sample"
results.upload_to_hub(owner="team13")


ValueError: shapes (87,277) and (2048,277) not aligned: 277 (dim 1) != 2048 (dim 0)